# Algorithmen

In [3]:
import sys
import os
import string
import time
import re
import datetime
import codecs
import itertools
import numpy as np
import sympy as sp
import pycurl
import matplotlib.pyplot as plt

from copy import deepcopy
from scipy.special import binom
from matplotlib import cm
from matplotlib.ticker import LinearLocator

# Ausgabe von Rechnername und Datum:
hostname = os.uname().nodename.split('.')[0].capitalize()
last_started = datetime.date.today()
print(f"Notebook wurde zuletzt gestartet {last_started} am Rechner {hostname}.")

Notebook wurde zuletzt gestartet 2025-06-02 am Rechner Fullmacstudio.


In [2]:
# OMIT
# Selbstgebastelte Module:
from mf.lecture import write_code_snippet, write_function_snippet, MF_ROOTPATH
from mf.texutils import make_standalone_pdf, find_occurrences
from mf.binary_trees import OTHER_SIDE, BinaryTreeNode, BinaryTree, get_list_of_nodes, draw_binary_tree, simple_key_in_circle, adhoc_for_kd_tree, AVLTreeNode, AVLTree, key_n_length_in_circle
from mf.geo import npp, get_line_equation, get_solution_xy, get_normalvector, get_angle, get_normalized
from mf.picture import PstricksPicture
from mf.decorator import ps_decorator_factory

# Root folder:
ROOTPATH = '/Users/mfulmek/ucloud/books/dmti/'
TESTPATH = '/Users/mfulmek/ucloud/books/test/'

# Decorator:
ps_dmti = ps_decorator_factory(rootpath=ROOTPATH)
# Für Testzwecke:
ps_test = ps_decorator_factory(rootpath=TESTPATH)

# Name und Nummer des Files für das aktuelle Kapitel:
NUMBER = 6
SECTION = 'algorithms'
SOURCE = os.path.join(ROOTPATH, "snippets", f'{SECTION}_snippets.ipynb')

# Keyword-Arguments für die Funktionsaufrufe:
COMMON = {
    'rootpath' : ROOTPATH,
    'the_source' : SOURCE,
    'only_python' : False
}

ONLY_PYTHON = {
    'rootpath' : MF_ROOTPATH,
    'the_source' : SOURCE,
    'only_python' : True
}

# Ausgabe von Kapitel:
print(f"Snippets für Kapitel 2: Algorithmen.")

Snippets für Kapitel 2: Algorithmen.


In [ ]:
# MOODLE

## Euklidischer Algorithmus

In [3]:
def euclidean_algorithm(a,b):
    """Euklidischer Algorithmus zur Bestimmung des größten
    gemeinsamen Teilers der (ganzen) Zahlen a und b."""
    # Es sollte a > b > 0 gelten: Wir korrigieren
    # "fehlerhafte" Eingaben, falls erforderlich.
    if a == 0:
        return b
    if b == 0:
        return a
    if a < 0:
        a = -a
    if b < 0:
        b = -b
    if a < b:
        # Vertausche a und b, sodass a > b gilt: Achtung, die Zeilen
        # a = b
        # b = a
        # würden NICHT das gewünschte Ergebnis liefern!!! Möglich wäre:
        # dummy = a
        # a = b
        # b = dummy
        # Aber in Python geht das einfacher:
        a,b = b,a
    # Die folgende while-Schleife läuft scheinbar "für immer":
    # Wir springen aber natürlich nach endlich vielen Schritten
    # heraus, und zwar mit dem return-Befehl.
    while True:
        # Division mit Rest: 
        quotient = a // b # NICHT a / b: Ergebnis i.a. KEINE ganze Zahl!
        remainder = a % b # remainder = a modulo b, 
        # also a = quotient*b + remainder
        if remainder == 0:
            # Der letzte auftretende Rest ungleich Null ist b:
            return b
        # Implizites else: Im Fall remainder==0 sind wir ja aus der Funktion
        # (und daher auch aus der while-Schleife) mit dem return-Befehl
        # "herausgesprungen".

        # Vorbereitung der nächsten Division mit Rest:
        a = b
        b = remainder

In [4]:
write_function_snippet(euclidean_algorithm,
    **COMMON,
    the_caption=r'euclidean-algorithm',
    preamble = ''
)

\snippet{euclidean-algorithm}
Process euclidean-algorithm


## ```numpy```: Eine Sammlung äußerst nützlicher Funktionen

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
# Arrays (d.h.: Vektoren und Matrizen) mit numpy:
# Einfache Definition einer 2x2-Matrix, wo alle Einträge 0.0 sind: 
meine_matrix = np.zeros(4, dtype = float).reshape((2,2))
# Verändern der Einträge dieser Matrix:
meine_matrix[0][0] = 1
meine_matrix[0][1] = 2
meine_matrix[1][0] = 3
meine_matrix[1][1] = 4
print(meine_matrix)
# Alternative Definition einer 2x2-Matrix, Einträge hier Datentyp int:
# Zuerst wird der Vektor (1,2,3,4) der Länge 4 erzeugt, der dann in
# eine 2x2-Matrix "umgewandelt" wird (mit reshape):
meine_matrix = np.arange(1,5, dtype = int).reshape((2,2))
print(meine_matrix)
# Die "Dimensionen" einer Matrix erhält man wie folgt:
zeilen, spalten = meine_matrix.shape
print(f'Unsere Matrix hat {zeilen} Zeilen und {spalten} Spalten.')
# Multiplikation einer Matrix mit einem Vektor:
mein_vektor = np.arange(2, dtype = int) # kein "reshape" erforderlich! 
print('Multiplikation Matrix*Vektor:')
print(f'{meine_matrix}*{mein_vektor} = {meine_matrix.dot(mein_vektor)}')
print('Multiplikation Matrix*Matrix:')
print(f'{meine_matrix}*{meine_matrix} = {meine_matrix.dot(meine_matrix)}')

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'numpy-basics',
    **COMMON,
    the_caption = r'numpy-basics',
    preamble = ''
)

## product_lex
Erzeugung der Elemente eines kartesischen Produkts "der Reihe nach", in _lexikographischer_ Ordnung.

An diesem einfachen Beispiel illustrieren wir zugleich einige
Feinheiten von Python: Darum sind die Kommentare viel
umfangreicher als die eigentlichen Programmzeilen.

(Lassen Sie versuchsweise den Befehl ```deepcopy``` weg und schauen Sie sich an, was dann passiert.)

In [ ]:
def product_lex(maxima):
    """Erzeuge das kartesische Produkt
    {1,...,maxima[0]} x {1,...,maxima[1]} x ..."""
    #@ An diesem einfachen Beispiel illustrieren wir zugleich einige
    #@ Feinheiten von Python: Darum sind die Kommentare viel
    #@ umfangreicher als die eigentlichen Programmzeilen.
    # Anzahl der Faktoren des kartesischen Produkts:
    dim = len(maxima)
    # Lexikographisch kleinstes Element in diesem
    # kartesischen Produkt ist (mathematisch) das Tupel (1,1, ..., 1),
    # in Python die Liste [1,1,...,1] (list und tuple sind in Python
    # _verschiedene_ Datentypen!):
    current_tuple = [1]*dim
    # deepcopy verwenden: Erzeuge eine KOPIE von current_tuple
    # und gib diese als erstes Element in die Liste von Listen,
    # die in der Folge zum gesamten kartesischen Produkt erweitert
    # wird:
    cart_prod = [ deepcopy(current_tuple) ]
    
    def find_last_pos(n,ct,tm):
        """Hilfsfunktion innerhalb der Funktion product_lex:
        Sie soll aufgerufen werden mit n=dim, ct=current_tuple
        und tm=the_maxima."""
        # Suche die "am weitesten rechts stehende" Komponente
        # des Tupels, die kleiner ist als das Maximum für diese
        # Komponente (die also "hochgezählt" werden kann):
        for i in range(n-1,-1,-1):
            # Range von n-1 bis 0 in absteigender Reihenfolge
            if ct[i] < tm[i]:
                return i
        return -1

    # Der "walrus-operator := " liefert als _Ergebnis_ den zugewiesenen
    # Wert (d.h.: Das, was rechts von ":=" steht): Damit können wir die
    # Abbruchbedingung für die while-Schleife so schreiben:
    while (last_pos := find_last_pos(dim, current_tuple, maxima)) > -1:
        # Komponente last_pos kann "hochgezählt" werden ...
        current_tuple[last_pos] += 1
        # ... und alle Komponenten "links von last_pos" werden auf 1 gesetzt:
        current_tuple[last_pos+1:] = [1]*(dim-last_pos-1)
        # Nicht vergessen: KOPIE erzeugen mit deepcopy!
        cart_prod.append(deepcopy(current_tuple))
    
    # Wenn wir damit fertig sind, geben wir die erzeugte
    # Liste zurück:
    return cart_prod

In [ ]:
write_function_snippet(product_lex,
    **COMMON,
    the_caption=r'Erzeugung eines kartesischen Produktes in lexikographischer Ordnung',
    preamble = ''
)

## walrus-operator

Der "Walross-Operator" wurde in Python 3.8 eingeführt:

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
# Die folgende (auskommentierte) Zeile wäre ein Syntax-Fehler ...
# print(f'Der Ausdruck ergibt "(a = 2) > 7)" ergibt den Wert {(a = 2) > 7}')
# ... denn die _Zuweisung_ "a=2" ergibt keinen Wert (d.h., im Python-Code
# "(a=2) > 7" steht links von ">" einfach _nichts_).
# Der "Walross-Operator ":=" funktioniert "wie eine Zuweisung", ergibt aber
# einen Wert (nämlich das, was rechts von ":=" steht):
print(f'Der Ausdruck ergibt "(a:= 2) > 7)" ergibt den Wert {(a:= 2) > 7}')
print(f'Denn die Zuweisung "a:= 2" mit Walross-Operator ":=" ...')
print(f'... ERGIBT den Wert {(a:= 2)}')

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'walrus-operator',
    **COMMON,
    the_caption = r'Der \anf{walrus operator} (ab Python Version 3.8)',
    preamble = ''
)

## product_with_itertools

 Wie so viele andere nützliche Algorithmen und Routinen ist auch die Erzeugung
 des kartesischen Produkts in Python bereits in einem Modul (nämlich in ```itertools```) ausprogrammiert,
 das man nur importieren muß.
 
     itertools.product(the_list,len(the_list))

liefert einen sogenannten _Iterator_:
 Das heißt, daß _nicht_ die gesamte Liste aller Tupel auf einmal erzeugt
 wird (was ja einen großen Speicherplatzverbrauch bedeuten würde), sondern
 eine _Funktion_}, die (z.B. in einer for-Schleife) die Tupel _eines
 nach dem anderen_ liefert.
 
 Wir sehen hier auch ein weiteres nützliches Merkmal von Python: Man kann
 einer Funktion ohne weiteres eine _Funktion_ (hier: ```the_func```) als Argument
 übergeben!

In [ ]:
def product_with_itertools(the_list, the_func):
    """Verwende die Python-Bibliotheksfunktion itertools.product()
    zur Erzeugung des cartesischen Produkts: """
    #@ Wie so viele andere nützliche Algorithmen und Routinen ist auch die Erzeugung
    #@ des kartesischen Produkts in Pythn bereits in einem Modul (itertools) ausprogrammiert,
    #@ das man nur importieren muß.
    #@ 
    #@ \verb|itertools.product(the_list,len(the_list))| liefert einen Iterator:
    #@ Das heißt, daß \EM{nicht} die gesamte Liste aller Tupel auf einmal erzeugt
    #@ wird (was ja einen großen Speicherplatzverbrauch bedeuten würde), sondern
    #@ eine \EM{Funktion}, die (z.B. in einer for--Schleife) die Tupel \EM{eines
    #@ nach dem anderen} liefert.
    #@ Wir sehen hier auch ein weiteres nützliches Merkmal von Python: Man kann
    #@ einer Funktion ohne weiteres eine \EM{Funktion} (hier: \verb|the_func|) als Argument
    #@ übergeben!
    # Vorbereiten: itertools.product übernimmt _beliebig viele Listen_ als
    # Argumente; wir erzeugen also zunächst eine Liste von Listen (der
    # Befehl range(1,i+1) entspricht der Liste [1,2,...,i]) ...
    arguments = [list(range(1,i+1)) for i in the_list]
    # ... und "entfernen die äußersten Listenklammern [ ... ]" durch
    # "*" bei der Übergabe von arguments: Das muß man sich so vorstellen
    # product(*[l_1, l_2, ..., l_m]) -> product(l_1, l_2, ..., l_m)
    # (denn itertools.product "erwartet" die Argumente in dieser Form):
    for pi in itertools.product(*arguments):
        # Wende die Funktion the_func auf das aktuelle Tupel an:
        # Ein weiteres, sehr nützliches Merkmal von Python ist die
        # problemlose Übergabe einer Funktion als Argument an eine andere
        # Funktion!
        the_func(pi)

In [ ]:
write_function_snippet(product_with_itertools,
    **COMMON,
    the_caption=r'product-with-itertools',
    preamble = ''
)

## Aufruf von ```product_with_itertools``` mit einer Lambda--Funktion.

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
# Aufruf der Funktion product_with_itertools:
# Als Argument the_func könnte z.B. zum Beispiel "print" übergeben werden -
# dann würden einfach alle erzeugten Tupel des cartesischen Produkts
# ausgegeben. Statt einer zuvor definierten Funktion kann aber auch eine
# "lambda function" übergeben werden, das ist eine kleine "namenlose"
# Funktion, die mehrere Argumente übernehmen kann, die aber nur aus einer
# _einzigen_ Anweisung bestehen darf; siehe das folgende Beispiel.
# (Der Python-Befehl sum(l) entspricht l[0] + le[1] + ... +l[len(l)-1].)
product_with_itertools(
    [2,3],
    lambda x : print(f'Summe über {x} ergibt {sum(x)}')
)

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'call-product-with-itertools',
    **COMMON,
    the_caption = r'Aufruf von \pythoncode{product\_with\_itertools} mit einer Lambda--Funktion.',
    preamble = ''
)

## find_last_pos

Hier noch einmal die Funktion, die in ```product_lex``` als _lokale_ (d.h., in der Funktion ```product_lex``` selbst definierte und "außerhalb" nicht sichtbare) Hilfsfunktion definiert wurde.

In [ ]:
def find_last_pos(current_tuple, maxima):
    """Hilfsfunktion für generate_product_lex."""
    # Suche erste Position "von rechts beginnend", die "erhöht" werden kann:
    # for-Schleife von n-1 (Index der letzten Komponente) bis 0, absteigend.
    for i in range(len(current_tuple)-1,-1,-1):
        if current_tuple[i] < maxima[i]:
            return i
    # Keine gültige Position gefunden:
    return -1

In [ ]:
write_function_snippet(find_last_pos,
    **COMMON,
    the_caption=r'find-last-pos',
    preamble = ''
)

## generate_product_lex

Noch einmal das Verfahren zur Erzeugung eines kartesischen Produkts _in lexikographischer Ordnung_, diesmal aber (computertechnisch viel besser!) mit einem _Generator_: Das ist eine Funktion, die einen _Iterator_ erzeugt. Zum Beispiel ist ```range```ein Generator, und der Aufruf dieses Generators ```range(0,10,2)``` erzeugt einen ```Iterator```, der in einer ```for```-Schleife der Reihe nach die Zahlen $0,2,4,6,8$ liefern würde.

In [ ]:
def generate_product_lex(maxima):
    """Generator, der einen Iterator für das Kartesische Produkt
    in _lexikographischer_ Ordnung liefert: Das Argument dieser Funktion
    ist eine Liste von natürlichen Zahlen m_1, m_2, ... m_n.
    Wir erzeugen also alle Elemente des Produkts [m_1]x[m_2]x...x[m_n],
    aber nicht "alle auf einmal, in einer langen Liste", sondern "on demand",
    also immer nur ein einziges Tupel, wenn es angefordert wird.
    
    Dabei achten wir auch auf zwei "Grenzfälle"."""
    # Grenzfall 1:
    # Wenn das kartesische Produkt _leer_ keinen einzigen Faktor (anders
    # gesagt: "0 Faktoren") enthält, dann ist das Ergebnis das leere Tupel
    # (mit 0 Komponenten)
    if not maxima:
        # Mathematisch: Leeres Tupel, in Python: Leere Liste.
        return []
        
    # Implicit else: Wenn wir nicht bereits aus der Funktion mit
    # "return" aus der vorigen if-Abfrage "herausgesprungen" sind.
    # Grenzfall 2:
    # Wenn ein Faktor des Produkts die leere Menge ist,
    # ist auch das ganze kartesische Produkt leer!!!
    if 0 in maxima:
        # Dieser "normale" return-Befehl bedeutet: Der vom "Generator"
        # erzeugte "Iterator" endet hier - er liefert also kein einziges
        # Tupel.
        return

    # Implicit else: Wenn wir nicht bereits aus der Funktion mit
    # "return" aus den vorigen if-Abfragen "herausgesprungen" sind.
    # Normalfall:
    dim = len(maxima)
    # Das ist das erste ("kleinste") Tupel (in Python: Datentyp Liste,
    # denn ein "tuple" in Python ist "immutable", d.h., wir können seine
    # Komponenten später nicht mehr verändern) in lexikographischer
    # Ordnung:
    current_tuple = [1]*dim

    # Mit dem Walross-Operator wird die Sache einfach:
    while (last_pos := find_last_pos(current_tuple, maxima)) > -1:
        # Der Befehl yield macht aus der Funktion einen "Generator".
        # Man muß sich das so vorstellen: Im "Iterator", der von dem
        # "Generator" erzeugt wird, funktioniert "yield" wie "return";
        # nur wird nach dem "yield" die Funktion nicht beendet, sondern
        # der "Iterator" setzt bei der nächsten "Aufforderung, einen
        # Wert zu liefern", unmittelbar nach diesem "yield" wieder fort.
        # (Die "Aufforderung, einen Wert zu liefern" erfolgt in der
        # typischen Anwendung einer for-Schleife _unsichtbar_ für die
        # Programmiererin.)
        yield current_tuple
        
        current_tuple[last_pos] += 1
        current_tuple[last_pos+1:] = [1]*(dim-last_pos-1)
        
        last_pos = find_last_pos(current_tuple, maxima)
    
    # Abschließend nochmals "yield", um auch die letzte Liste auszugeben:
    yield current_tuple
    # Hier endet der vom "Generator" erzeugte "Iterator". (Den "return"
    # Befehl könnten wir auch weglassen, weil er "implizit" am Ende der
    # Funktionsdefinition (englisch: function body) ergänzt wird.)
    # In der typischen Anwendung für eine for-Schleife wird die Schleife
    # nun beendet.
    return

In [ ]:
write_function_snippet(generate_product_lex,
    **COMMON,
    the_caption=r'generate-product-lex',
    preamble = ''
)

## rank_product_lex

Wenn man die ganzen Kommentare weglässt und Zeilen zusammenfasst, dann hat diese Funktion nur drei Zeilen!

In [ ]:
def rank_product_lex(the_tuple, the_maxima):
    """Bestimme die _Nummer_ des gegebenen Tupels
    innerhalb des cartesischen Produkts
    [the_maxima[0]] x [the_maxima[1]] x ..."""
    #@ (Wenn man die ganzen Kommentare weglässt und Zeilen
    #@ zusammenfasst, dann hat diese Funktion nur drei Zeilen!)
    # Für unsere Zwecke ist es besser, "mit dem Zählen bei
    # Null zu beginnen", auch wenn das nicht unserer Gewohnheit
    # entspricht: Wenn wir konsequent "mit dem Zählen bei Null
    # beginnen", dann machen wir auch alle Einträge unseres Tupels
    # um 1 kleiner; wir führen diese einfache "Transformation"
    # durch und illustrieren zugleich die "list comprehension",
    # eine sehr nützliche syntaktische Feinheit von Python:
    tup = [x-1 for x in the_tuple]
    # Nun wandeln wir das Resultat in einen numpy-Vektor
    # mit Datentyp int (englisch "integer": "ganze Zahl")
    # um (daß tup nach dieser Umwandlung ein anderer
    # Datentyp ist, ist in Python kein Problem - jedenfalls
    # kein _syntaktisches_ ;-):
    tup = np.array(tup)
    # Das könnte man auch in eine Zeile zusammenfassen:
    # tup = np.array([x-1 for x in the_tuple], dtype=int)

    # Drehe die Liste the_maxima um, hänge vorne die Zahl 1
    # an, und verwandle die Liste in einen numpy-Vektor
    # mit Datentyp int:
    aux = np.array([1]+the_maxima[::-1], dtype=int)
    # Überschreibe den eben erzeugten Vektor mit den
    # "kumulativen Produkten", das ist der Vektor
    # (aux[0], aux[0]*aux[1], aux[0]*aux[1]*aux[2], ...):
    aux = np.cumprod(aux)
    # Überschreibe den eben erzeugten Vektor mit dem
    # "umgedrehten" Vektor (dessen Elemente in umgekehrter
    # Reihenfolge erscheinen), wobei das erste Element
    # weggelassen wird ("Slicing" funktioniert bei numpy-arrays
    # genauso wir bei Python-Listen):
    aux = np.flip(aux)[1:]
    # Das könnte man auch in eine Zeile zusammenfassen:
    # aux=np.flip(np.cumprod(np.array([1]+the_maxima[::-1],dtype=int)))[1:]

    # Mathematisch ist die "Nummer" (wenn wir bei Null zu zählen
    # beginnen) gleich dem _inneren Produkt_ von tup und aux
    # (es lohnt sich nachzudenken, _warum_ das so ist!); da
    # wir aber zunächst bei unserer Gewohnheit bleiben und
    # das Zählen bei Eins beginnen wollen, addieren wir 1:
    return np.inner(tup, aux) + 1

In [ ]:
write_function_snippet(rank_product_lex,
    **COMMON,
    the_caption=r'rank-product-lex',
    preamble = ''
)

## Das Unranking von kartesischen Produkten in lexikographischer Ordnung.

In [ ]:
def unrank_product_lex(the_number, the_maxima):
    """Bestimme aus der gegebenen Nummer (mit 1 beginnend)
    das entsprechende Tupel in der lexikographischen
    Auflistung des cartesischen Produkts
    [the_maxima[0]] x [the_maxima[1]] x ...
    """
    # Wie in der rank-Funktion ist es besser, "mit dem Zählen bei
    # Null zu beginnen":
    the_number-= 1
    # (Äquivalent: "the_number = the_number - 1".)
    
    # Wir beginnen mit dem leeren Tupel:
    the_tuple = []
    for divisor in the_maxima[::-1]:
        # Division mit Rest: a % b für zwei ganze Zahlen
        # a, b liefert a modulo b.
        integer_remainder = the_number % divisor
        # Wir bleiben (noch) bei der Gewohnheit, das Zählen
        # bei Eins zu beginnen:
        the_tuple = [integer_remainder + 1]+the_tuple
        # Das könnte man auch in eine Zeile zusammenfassen:
        # the_tuple = [the_number % divisor + 1]+the_tuple
        
        # ACHTUNG, würde man hier
        #    "the_number / divisor"
        # schreiben, dann erhielte man eine Fließkommazahl
        # (Datentyp float), also i.a. KEINE ganze Zahl!!!
        the_number = the_number // divisor
    
    return the_tuple

In [ ]:
write_function_snippet(unrank_product_lex,
    **COMMON,
    the_caption=r'Das Unranking von cartesischen Produkten in lexikographischer Ordnung.',
    preamble = 'import numpy as np'
)

## Test ``$\text{rank}\circ\text{unrank} = \text{identity}$'' für cartesische Produkte in lexikographischer Ordnung.


In [ ]:
def test_product_lex(the_maxima):
    """Eine einfacher (aber gründlicher;-) Test,
    ob das Ranking/Unranking richtig funktioniert"""
    # Python hat zwar eine eingebaute Funktion sum(), die die
    # Elemente einer Liste _aufsummiert_, aber keine analoge
    # Funktion prod() (jedenfalls nicht in Python-Versionen < 3.8),
    # die die Elemente einer Liste _aufmultipliziert_: Wir können
    # uns aber mit der ensprechenden numpy-Funktion behelfen.
    cardinality = np.prod(np.array(the_maxima, dtype=int))
    # Schleife über alle Zahlen von 1 bis zur Kardinalität des
    # cartesischen Produkts:
    for i in range(1, cardinality+1):
        # Es müßte "rank o unrank = identität" gelten - wenn nicht,
        # geben wir eine Fehlermeldung aus:
        if i != rank_product_lex(
            unrank_product_lex(
                i,
                the_maxima
            ),
            the_maxima
        ):
            # Eine der (mehreren!!) Arten, einen STRING (also eine
            # Kette von Buchstaben) _formatiert_ auszugeben: Was in
            # geschwungenen Klammern steht, wird durch die entsprechende
            # _string representation_ ersetzt.
            print(f'Fehler im kart. Produkt {the_maxima} bei Nummer {i}!')
            return False
    
    print(f'Alles bestens: (rank o unrank) = identity gilt für {the_maxima}!')
    return True

In [ ]:
write_function_snippet(test_product_lex,
    **COMMON,
    the_caption=r'Test ``$\text{rank}\circ\text{unrank} = \text{identity}$'' für cartesische Produkte in lexikographischer Ordnung.',
    preamble = 'import numpy as np'
)

## "Generischer" Test für Ranking/Unranking.

Das Wort "generisch" bedeutet "allgemein, nicht spezifisch": In unserem Zusammenhang ist damit gemeint, dass die Funktion nicht nur Ranking/Unranking für _kartesische Produkte_ überprüft, sondern für "beliebige kombinatorischer Objekte".

In [ ]:
def generic_rank_unrank_test(
    description,
    rank_func,
    unrank_func,
    cardinality
):
    """Ein einfacher (aber gründlicher;-) _generischer_ Test,
    ob das Ranking/Unranking richtig funktioniert: description ist
    eine Variable, die den Ranking/Unranking-Funktionen rank_func
    und unrank_funk übergeben wird, und cardinality ist die Anzahl
    der erzeugten Objekte."""
    # Schleife über alle Zahlen, die als Rang eines description-Objekts
    # auftreten (sollen):
    for i in range(cardinality):
        # Es müßte "rank o unrank = identität" gelten - wenn nicht, geben
        # wir eine Fehlermeldung aus:
        if i != rank_func(unrank_func(i,description),description):
            print(f'Oops: Fehler für {description} bei Nummer {i}!')
            return False
    
    print(f'Ok: (rank o unrank) = identity gilt für {description}!')
    return True

In [ ]:
write_function_snippet(generic_rank_unrank_test,
    **COMMON,
    the_caption=r'``Generischer'' Test für Ranking/Unranking.',
    preamble = ''
)

## Generator: Gray-Code für kartesisches Produkt.

In [ ]:
def product_gray_code(the_maxima):
    """Erzeuge Gray-Code des cartesischen Produkts: Diesmal
    beginnen wir mit dem Zählen gleich bei Null, betrachten also
    {0,...,m_1-1} x {0,...,m_2-1} x ... x {0,...,m_n-1}.
    ACHTUNG: Hier muß min(the_maxima) > 1 gelten! (Faktoren der
    Mächtigkeit 1 sind ja "in allen Tupeln konstant", und
    Faktoren der Mächtigkeit 0 ergeben ein leeres Produkt;
    das ist also keine starke Einschränkung.)
    """
    if min(the_maxima) <= 1:
        # Diesen Fall behandeln wir hier nicht.
        print('Wir behandeln nur Produkte von Faktoren S mit |S|>1!')
        return
    
    dim = len(the_maxima)
    # Erstes Tupel:
    current_tuple = np.zeros(dim, dtype=int)
    # Vektor, der die "Richtungen" anzeigt, in die
    # die Komponenten des Tupels während des Algorithmus
    # bewegt werden:
    directions = np.ones(dim, dtype=int)
    # Vektor, der die "aktiven" Koordinaten anzeigt:
    active_flags = np.ones(dim, dtype=int)
    
    finished = False
    while not finished:
        yield current_tuple
        # Die numpy-Funktion nonzero(v) liefert array der Indices
        # der Elemente von v, die ungleich Null sind; genauer gesagt:
        # Ein _Tupel_ von arrays von Indizes - für jede Dimension
        # von v ein Index-Array; unser v is eindimensional, wir brauchen
        # also einfach das ERSTE Element (mit Index 0) dieses Tupels:
        active_positions = np.nonzero(active_flags)[0]
        # Wenn es keine aktiven Koordinaten mehr gibt: Fertig!
        if len(active_positions) == 0:
            return
        # Die größte aktive Koordinate wird in die entsprechende
        # Richtung (+/- 1) verändert:
        pos = np.max(active_positions)
        current_tuple[pos] += directions[pos]
        # Wenn wir an die "oberen/unteren Grenzen" gestoßen sind,
        # kehren wir die Richtung um und setzen die Koordinate auf
        # "nicht aktiv":
        if current_tuple[pos]==0 or current_tuple[pos]==the_maxima[pos]-1:
            directions[pos] = -directions[pos]
            active_flags[pos] = 0
        # Setze alle active-flags ab pos+1 auf 1: Die entsprechenden
        # Koordinaten können im nächsten Schleifen-Durchlauf verändert
        # werden. - Zuweisung einer Zahl zu numpy-array-slice ist möglich!
        active_flags[pos+1:] = 1

In [ ]:
write_function_snippet(product_gray_code,
    **COMMON,
    the_caption=r'Generator: Gray--Code für cartesisches Produkt.',
    preamble = 'import numpy as np'
)

## Test: Gray-Code für kartesisches Produkt.

In [ ]:
def test_product_gray_code(the_maxima):
    # Erster Test: Haben zwei aufeinanderfolgende Tupel tatsächlich
    # Abstand 1 (in der 1-Norm)
    for i,t in enumerate(product_gray_code(the_maxima)):
        if i==0:
            v = np.copy(t)
        else:
            # Da alle Koordinaten von t-v GANZZAHLIG sind, ist die
            # 1-Norm von (t-v) genau dann gleich 1, wenn das für die
            # "normale" euklidische Norm gilt:
            if np.linalg.norm(t-v) != 1:
                print(f'Mist: 1-Norm von ({t} - {v}) ungleich 1!')
                return False
            # NICHT "v=t": Dann würde nämlich Variablen v und t auf
            # DASSELBE numpy-array verweisen!
            v[:] = t
    # Zweiter Test: Liefert der Gray-Code dieselben Tupel wie der
    # lexikographische Algorithmus
    return sorted([t for t in product_lex(the_maxima)]) == \
sorted([list(t+1) for t in product_gray_code(the_maxima)])

In [ ]:
write_function_snippet(test_product_gray_code,
    **COMMON,
    the_caption=r'Test: Gray--Code für cartesisches Produkt.',
    preamble = 'import numpy as np'
)

## Ranking: Gray-Code für cartesisches Produkt.

In [ ]:
def rank_product_gray_code(the_tuple, the_maxima):
    """Das Ranking für den Gray-Code eines cartesischen Produkts:
    Liefere für gegebenes Tupel the_tupel die "Nummer" in der
    Aufzählung, die durch product_gray_code erzeugt wird.
    """
    rank = 0
    dim = len(the_tuple)
    for i in range(dim):
        if rank % 2:
            remainder = the_maxima[i] - the_tuple[i] - 1
        else:
            remainder = the_tuple[i]
        rank = rank*the_maxima[i] + remainder
    return rank

In [ ]:
write_function_snippet(rank_product_gray_code,
    **COMMON,
    the_caption=r'Ranking: Gray--Code für cartesisches Produkt.',
    preamble = ''
)

## Unranking: Gray-Code für cartesisches Produkt.

In [ ]:
def unrank_product_gray_code(the_number, the_maxima):
    """Das Unranking für den Gray-Code eines cartesischen Produkts:
    Liefere für gegebenes Zahl the_number das entsprechende Tupel in
    der Aufzählung, die durch product_gray_code erzeugt wird."""
    dim = len(the_maxima)
    the_tuple = np.zeros(dim, dtype=int)
    for i in range(dim-1,-1,-1):
        the_tuple[i] = the_number % the_maxima[i]
        the_number//= the_maxima[i]
        if the_number % 2:
            the_tuple[i] = the_maxima[i] - the_tuple[i] - 1
    return the_tuple

In [ ]:
write_function_snippet(unrank_product_gray_code,
    **COMMON,
    the_caption=r'Unranking: Gray--Code für cartesisches Produkt.',
    preamble = ''
)

## Generator, der einen (klassischen) Gray-Code-Iterator liefert.
 Die Binärzahlen eines Gray-Codes interpretieren wir hier als
 _charakteristische Funktionen_ (siehe Definition im Skriptum) von Teilmengen.
 Der Datentyp ```set``` in Python entspricht dem mathematischen Begriff
 einer (natürlich endlichen!) Menge.

In [ ]:
def classical_gray_code(maximum=8):
    """Generator-Funktion: Liefert Iterator, der einen (klassischen)
    Gray-Code erzeugt"""
    #@ Die Binärzahlen eines Gray-Codes interpretieren wir hier als 
    #@ charakteristische Funktionen (siehe \dfnref{charakteristische-funktion}) von Teilmengen.
    #@ Der Datentyp \pythoncode{set} in Python entspricht dem mathematischen Begriff
    #@ einer (natürlich endlichen!) Menge.
    subset = set()
    # Als Binärzahl ist das also 0000...
    while maximum > 0:
        maximum-= 1
        # Der Befehl "yield" wirkt wie "return", macht aber aus der Funktion
        # einen "Generator", die einen "Iterator" liefert:
        yield subset
        element_to_flip = (min(subset) + 1) if (len(subset) % 2) else 1
        # Die Symmetrische Differenz zweier Mengen A und B ist
        #    (A vereinigt B) ohne (A geschnitten mit B):
        subset = subset.symmetric_difference(set([element_to_flip]))

In [ ]:
write_function_snippet(classical_gray_code,
    **COMMON,
    the_caption=r'Generator, der einen (klassischen) Gray--Code-Iterator liefert.',
    preamble = ''
)

## Funktion, die eine Teilmenge natürlicher Zahlen in ihre charakteristische Funktion umwandelt.


In [ ]:
def set_to_characteristic_function(the_set, the_width):
    """Convert the_set (supposed to be a subset of
    [the_width] = {1,2,...,the_width}) to its characteristic
    function"""
    char_func = [0]*the_width
    for x in the_set:
        # Nicht vergessen: Indexierung beginnt bei 0.
        char_func[x-1] = 1
    return char_func

In [ ]:

write_function_snippet(set_to_characteristic_function,
    **COMMON,
    the_caption=r'Funktion, die eine Teilmenge natürlicher Zahlen in ihre charakteristische Funktion umwandelt.',
    preamble = ''
)

## Funktion, die einen (klassischen) Gray-Code als Binärzahlen ausgibt.

In [ ]:
def print_classical_gray_code(the_width):
    """Print a (classical) Gray code of width the_width"""
    for dum in classical_gray_code(2**the_width):
        char_func = set_to_characteristic_function(dum,the_width)
        # Der Datentyp string hat eine "member function" join,
        # die eine Liste von strings "aneinanderhängt", mit dem
        # "aufrufenden Objekt" (hier der leere String "") als
        # "Verbindungs-String". Z.B. liefert
        #    "*".join(['a','b','c'])
        # den String
        #    "a*b*c".
        binary_string = "".join([str(i) for i in char_func])
        print(binary_string)

In [ ]:
write_function_snippet(print_classical_gray_code,
    **COMMON,
    the_caption=r'Funktion, die einen (klassischen) Gray--Code als Binärzahlen ausgibt.',
    preamble = ''
)

## Ein Iterator als Klasse, am Beispiel der $k$-elementigen Teilmengen einer $n$-elementigen Menge.

Eine _Klasse_ kann man sich als einen Datentyp vorstellen, für den man selbst definieren kann, aus welchen Variablen er sich zusammensetzt und welche Funktionen und Operationen dafür möglich sind.
Für einen _Iterator_ müssen wir die Funktionen 
* ```__iter__``` : Initialisiere den Iterator
* ```__next__``` : Liefere das nächste Objekt (üblicherweise auf "implizite" Anforderung, z.B. in einer ```for```-Schleife)
implementieren.

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!

# Eine selbst definierte Klasse (englisch: class) in Python
class subsets_colex_iterator:
    """Iterator: k--elem. Teilmengen von [n] in co-lexikographischer
    Reihenfolge"""

    # Eine Klasse "braucht" (in der Regel) eine "Initialisierung":
    def __init__(self, n, k):
        if n < 0 or k > n:
            print('Sehr lustig:-(')
            n = k = 0
        self.n = n
        self.k = k

    # Die Funktionen __iter__ und __next__ implementieren die
    # "Iterator-Eigenschaft" der Klasse:
    def __iter__(self):
        # Nicht vergessen: Listen-Indizierung beginnt bei Null, range(k)
        # liefert {0,1,...,k-1}; die erste Teilmenge in der
        # co-lexikographischen Ordnung ist {1,...,k} - wir speichern
        # aber _ein Element mehr ab_, aus Gründen, die weiter unten klar
        # werden ...
        self.current_subset = [i+1 for i in range(self.k+1)]
        # ... genauer gesagt: Wir schreiben ans Ende n+1 als
        # "Ende-Markierung" ...
        self.current_subset[self.k] = self.n+1
        # ... und erstes Element 0 als Flag, daß wir grade erst begonnen
        # haben:
        self.current_subset[0]=0
        
        # Plan: self_current_subset ist IMMER eine Liste von aufsteigend
        # geordneten k+1 positiven ganzen Zahlen, deren letztes Element
        # IMMER gleich n+1 ist.
        return self

    def __next__(self):
        # Wenn die Teilmenge self.current_subset gleich {n-k+1,...,n} ist,
        # dann geht's nicht mehr weiter:
        if self.k == 0 or self.current_subset[0] >= self.n-self.k + 1:
            raise StopIteration
        else:
            # Suche das erste Element, das um mindestens 2 kleiner ist als
            # sein Nachfolger (wenn wir bis hierher gekommen sind, _muß_ es
            # so ein Element geben - spätestens bei j = n, dann gilt die
            # Abbruchbedingung für die "Endmarkierung".)
            j = 0
            while self.current_subset[j]+1 >= self.current_subset[j+1]:
                j+= 1
            # Dieses Element erhöhen wir um 1 ...
            self.current_subset[j]+= 1
            # ... und setzen alle vorhergehenden Elemente auf den 
            # "co-lex-Minimalwert":
            self.current_subset[:j] =  [i+1 for i in range(j)]
            
            return self.current_subset[:-1]

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'subsets-class-iterator',
    **COMMON,
    the_caption = r'Ein Iterator als Klasse, am Beispiel der $k$--elementigen Teilmengen.',
    preamble = ''
)

## Verwendung einer Iterator-Klasse.

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!

# Create an object
object = subsets_colex_iterator(7,3)

# Create an iterable from the object
iterator = iter(object)

# Use the iterator in a for loop
for dum in iterator:
    print(dum)

# Try to call next() (this will fail!)
try:
    print(next(iterator))
except StopIteration:
    print('Game over!')

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'use-subsets-class-iterator',
    **COMMON,
    the_caption = r'Verwendung einer Iterator--Klasse.',
    preamble = ''
)

## Ranking für Teilmengen in co-lexikographischer Ordnung.

In [ ]:
def rank_colex_subsets(subset, k=None):
    r"""Der Parameter k ist hier an sich überflüssig - es muß ja "implizit"
    k = len(subset) gelten -, wir geben ihn hier aber dazu, damit wir
    unseren "generischen rank/unrank-Test" verwenden können.
    Das Ranking basiert auf der Beobachtung:
    rank([v_1,\dots,v_k]) = #(k-Teilmengen von {1,...,v_k-1})
    PLUS rank([v_1,\dots,v_{k-1}])"""
    # Die eingebaute Funktion binom (aus Modul scipy.special) liefert
    # float-Werte; die Summe dieser Werte ist daher auch vom Typ float:
    # Wir wandeln sie also in Datentyp int um.
    return int(sum([binom(v_k-1,k+1) for k,v_k in enumerate(subset)]))

In [ ]:
write_function_snippet(rank_colex_subsets,
    **COMMON,
    the_caption=r'Ranking für Teilmengen in co--lexikographischer Ordnung.',
    preamble = '# from scipy.special import binom'
)

## Unranking für Teilmengen in co-lexikographischer Ordnung.

In [ ]:
def unrank_colex_subsets(num, k):
    """Bestimme die Teilmenge num in der colex-Aufzählung
    aller k-elementigen Teilmengen: Das Lustige ist, daß wir
    hier nicht angeben müssen, von welcher Obermenge wir
    die Teilmengen betrachten!"""
    subset = [0]*k
    for i in range(k,0,-1):
        # Nummer num=0 ist sofort klar: Die Positionen
        # 0 bis i-1 werden mit 1,2,...,i belegt!
        if num == 0:
            subset[:i] = [j for j in range(1,i+1)]
            return subset
        # Implicit else: Suche die größte Zahl p sodaß binom(p,i) <= num.
        p = i
        # Die eingebaute Funktion binom (aus Modul scipy.special) liefert
        # float-Werte; wir wandeln das in integer um:
        binko = int(binom(p,i))
        while True:
            p+=1
            # binom(p+1,i) = binom(p,i)*(p+1)//(p-i); hier wieder
            # _ganzzahlige_ Division, also "//" statt "/" !
            newbinko = (binko*p)//(p-i)
            if newbinko > num:
                 # ... update num ...
                num-= binko
                # ... Element i (index i-1 in Python!) ist p
                subset[i-1] = p
                break
            else:
                binko = newbinko

    return subset

In [ ]:
write_function_snippet(unrank_colex_subsets,
    **COMMON,
    the_caption=r'Unranking für Teilmengen in co--lexikographischer Ordnung.',
    preamble = '# ??? from scipy.special import binom'
)

## Rekursive Erzeugung aller Permutationen einer Liste.
 Die Funktion ```perm_lex``` erzeugt durch eine _Rekursion_
 eine _lexikographisch geordnete_ Liste, die
 alle Permutationen der gegebenen Liste ```symbols```
 enthält. (Das Argument muß hier wirklich eine Liste sein
 und kein String, denn der Datentyp ```str``` hat keine ```.remove()```-Funktion).

In [ ]:
def perm_lex(symbols):
    """Gib die Permutation der Elemente von symbols in lexiko-
    graphischer Ordnung aus: Der lexikographischen Ordnung wird
    _implizit_ die Ordnung der Elemente in symbols zugrunde-
    gelegt. Rückgabewert ist eine Liste aller Permutationen
    von symbols, also eine Liste von Listen; z.B.
        perm_lex([1,2]) -> [[1,2],[2,1]]
    """
    #@ Die Funktion \verb|perm_lex| erzeugt durch eine \EM{Rekursion}
    #@ eine \EM{lexikographisch geordnete} Liste, die
    #@ alle Permutationen der gegebenen Liste \verb|symbols|
    #@ enthält. (Das Argument muß hier wirklich eine Liste sein
    #@ und kein String, denn Strings haben keine .remove()-Funktion).
    # Wenn die Liste leer ist oder nur ein
    # Element enthält, dann gibt es nur _eine_
    # Permutation (0! = 1! = 1):
    if len(symbols) <= 1:
        return [symbols]
    
    # Wir brauchen hier kein "else", da wir die
    # Funktion ja mit "return" verlassen, wenn die
    # Bedingung erfüllt ist: "Implizit" befinden
    # wir uns hier also in der "else-clause":
    
    
    # Rekursive Erzeugung, falls len(symbols) > 1: Der
    # Rückgabewert ist eine _Liste_ aller Permutationen,
    # wir bereiten diesen Rückgabewert einmal als
    # leere Liste vor, die wir im folgenden "anfüllen"
    # werden:
    permutations = []
    
    # Schleife über alle Elemente x der Liste symbols:
    for x in symbols:
        # Wir stellen eine Kopie von symbols OHNE x her ...
        without_x = deepcopy(symbols)
        without_x.remove(x)
        
        # ... und erzeugen die Liste aller Permutationen
        # dieser Kopie durch einen _rekursiven Aufruf_
        # der Funktion perm_lex, fügen für alle
        # so erzeugten Permutationen am Anfang das 
        # Element x an, und erweitern den Rückgabewert
        # permutations um die so konstruierte Liste aller
        # Permutationen von symbols, die mit x beginnen:
        permutations += [
            [x] + pi for pi in perm_lex(without_x)
        ]

    # Rückgabewert permutations enthält nun alle Permutationen
    # von symbols in lexikographischer Ordnung: Zuerst kommen die,
    # die mit dem ersten Element von symbols beginnen, dann die,
    # die mit dem zweiten Element von symbols beginnen, usw.
    return permutations

In [ ]:
write_function_snippet(perm_lex,
    **COMMON,
    the_caption=r'Rekursive Erzeugung aller Permutationen einer Liste.',
    preamble = '# ??? from copy import deepcopy'
)

## Experimentelle Mathematik mit einer Monte-Carlo-Simulation.
 Einfaches Beispiel für eine Monte-Carlo-Simulation:
 Erzeuge zufällig Permutationen von $\range n$ und zähle,
 für wieviele davon die Testfunktion ```test_permutation```
 den Wert ```True``` liefert. Zugleich ein kleines Beispiel dafür,
 wie das Formatieren von Strings in Python funktioniert. (Es
 gibt aber _mehrere_ Arten, Strings zu formatieren!)

In [ ]:
#import numpy as np
def monte_carlo_permutations(n, sample_size, test_permutation):
    """Erzeuge zufällig sample_size Permutationen von {1,2,...,n}
    und liefere die relative Häufigkeit der Permutationen, für die
    die Testfunktion test_permutation den Wert True liefert."""
    #@ Einfaches Beispiel für eine Monte---Carlo--Simulation:
    #@ Erzeuge zufällig Permutationen von $\range n$ und zähle,
    #@ für wieviele davon die Testfunktion \verb|test_permutation|
    #@ den Wert True liefert. Zugleich ein kleines Beispiel dafür,
    #@ wie das Formatieren von Strings in Python funktioniert. (Es
    #@ gibt aber \EM{mehrere} Arten, Strings zu formatieren!)
    nfact = np.math.factorial(n)
    print(f'MC-simulation: Generate {sample_size} random permutations')
    print(f'of {n} elements ({n}! = {nfact}) and count for how many')
    print(f'of them function "{test_permutation.__name__}" returns True:')
    # Erzeuge einen Zufallszahlen-Generator (Funktionalität der
    # Numerik-Bibliothek numpy)
    random_number_generator = np.random.default_rng()
    # Der Zufallszahlen-Generator erzeugt sample_size zufällig
    # gewählte ganze Zahlen im Intervall von 0 (inklusive) bis n!
    # (exklusive); in Form eines numpy-Arrays.
    random_integers = random_number_generator.integers(
        low=0,
        high=np.math.factorial(n),
        size=sample_size
    )
    count = 0
    for i in random_integers:
        if test_permutation(unrank_perm_lex(i, n)):
            count+= 1
    return count/sample_size

In [ ]:
write_function_snippet(monte_carlo_permutations,
    **COMMON,
    the_caption=r'Experimentelle Mathematik mit einer Monte--Carlo--Simulation.',
    preamble = ''
)

## Generator: Permutationen in der "Gray-Code-artigen" Aufzählung von Johnson und Trotter.

In [ ]:
def generate_johnson_trotter(n):
    """Generator: Erzeuge die Permutationen von {1,2,...,n}
    mit dem Johnson-Trotter-Algorithmus."""
    
    # Initialisierung:
    #  - Permutation pi (als numpy-array),
    #  - inverse Permutation pi_inv(ebenso als numpy-array), 
    #  - und "Bewegungsrichtungen" der "wandernden Elemente"
    # sind dargestellt als numpy-arrays der Länge n+2.
    # Indices 0 und n+1 dienen nur der "Begrenzung"; das
    # eigentliche Permutationswort ist im "slice 1:n+1")
    # gespeichert.
    # Die numpy-Funktion arange(n) liefert Werte von 0 bis
    # n-1 in einem numpy-array:
    pi = np.arange(n+2, dtype=int)
    # Index 0 enthält - ebenso wie n+1 - eine "Begrenzung",
    # die während des Algorithmus nicht verändert wird:
    pi[0] = n+1
    pi_inv = np.arange(n+2, dtype=int)
    # Die numpy-Funktion ones(n) liefert ein numpy-array
    # der Länge n, dessen Eintragungen alle gleich 1 sind.
    # numpy-Vektoren kann man mit Skalaren (hier: -1) multiplizieren
    # und addieren - wie sich das für Vektoren gehört;-)
    dirs = (-1)*np.ones(n+2, dtype=int)
    # Datentyp dtype=int für die Einträge ist hier WICHTIG, da
    # diese teils als INDIZES in arrays verwendet werden!
    
    # Aktive Elemente (das sind die, die durch das Permutationswort
    # "wandern"): Anfangs alle außer 1.
    active = [x for x in range(2,n+1)]
    
    # Solange es aktive Elemente gibt, durchlaufen wir die
    # while-Schleife: len(active) ist zwar genau besehen eine
    # int-Zahl und kein bool-Wahrheitswert, wird aber "automatisch"
    # als Wahrheitswert interpretiert: False genau dann, wenn 0,
    # sonst True.
    while len(active):
        # Ausgabe des aktuellen Permutationswortes
        # (das, wie gesagt, dem "slice 1:-1" entspricht):
        yield ' '.join([str(x) for x in pi[1:-1]])
        # print(' '.join([str(x) for x in pi[1:-1]]))
        
        # Das größte der aktiven Elemente soll "wandern" ...
        m = max(active)
        # ... und zwar in die "entsprechende Richtung":
        
        # j ist die Stelle, an der Element m aktuell steht ...
        j = pi_inv[m]
        # ... wir vertauscehn das Element an Stelle j (also m)
        # mit seinem "Nachbarn" (in Richtung dirs[m]) ...
        neighbour_j = j + dirs[m]
        pi[j] = pi[ neighbour_j  ]
        pi[ neighbour_j ] = m
        # (Diese Vertauschung entspricht einer Transposition tau!)
        # ... und passen die inverse Permutation entsprechend
        # an, damit wieder pi[pi_inv[j]] = j für j=1,2,...n gilt:
        pi_inv[m] = neighbour_j
        pi_inv[pi[j]] = j
        
        # Haben wir einen "Umkehrpunkt am Rand" erreicht
        # Das ist dann der Fall, wenn wir - in dieselbe Richtung
        # wie bisher weiter wandernd - auf ein Element > m stoßen
        # würden (zur Erinnerung: pi[0] = pi[n+1] = n+1): Dann drehen
        # wir die Richtung von m um und "deaktivieren" m.
        if m < pi[j+2*dirs[m]]:
            dirs[m] = -dirs[m]
            active.remove(m)
        
        # Nach dem "Schritt von m" lassen wir auch alle
        # Elemente größer als m (wenn es welche gibt) wieder
        # aktiv werden:
        active+= [x for x in range(m+1,n+1)]
    
    # Ausgabe des aktuellen Permutationswortes:
    yield ' '.join([str(x) for x in pi[1:-1]])    

In [ ]:
write_function_snippet(generate_johnson_trotter,
    **COMMON,
    the_caption=r'Generator: Permutationen in der ``Gray--Code--artigen'' Aufzählung von Johnson und Trotter.',
    preamble = 'import numpy as np'
)

## Inverse (also: Umkehrabbildung) einer Permutation.
 Die Inverse zu einer Permutation $\pi$ ist einfach die Umkehrabbildung:
 In unserer Python--Umsetzung ist wieder zu bedenken, daß die Indizierung
 von Listen _bei Null_ beginnt!


In [ ]:
def invert_pi(pi):
    """Sei pi = [pi_1, ..., pi_n] eine Permutation der ersten n
    natürlichen Zahlen, codiert in Einzeilen-Notation (Achtung: Das
    "Indizieren" in einer Liste beginnt bei 0): Berechne pi^(-1) (in
    derselben Einzeilen-Notation)."""
    #@ Die Inverse zu einer Permutation $\pi$ ist einfach die Umkehrabbildung:
    #@ In unserer Python--Umsetzung müssen wir wieder zu bedenken, daß die Indizierung
    #@ von Listen \EM{bei Null} beginnt!
    # Umkehrfunktion ist einfach:
    for i,pi_i in enumerate(pi, start = 1):
        # Indizierung beginnt bei Null - nicht vergessen!
        inverse[pi_i-1] = i
    return inverse

In [ ]:
write_function_snippet(invert_pi,
    **COMMON,
    the_caption=r'Inverse (also: Umkehrabbildung) einer Permutation.',
    preamble = ''
)

## Hilfsfunktion zur Bestimmung des Inversionswortes einer Permutation.


In [ ]:
def count_less_than_last(pi):
    """Hilfsfunktion: Wieviele Element der Liste sind echt kleiner als ihr
    letztes Element"""
    return sum([1 if pi_i < pi[-1] else 0 for pi_i in pi[:-1]])

In [ ]:
write_function_snippet(count_less_than_last,
    **COMMON,
    the_caption=r'Hilfsfunktion zur Bestimmung des Inversionswortes einer Permutation.',
    preamble = ''
)

## Rang (also Nummer) einer Permutation der ${\mathfrak S}_n$ in der Johnson-Trotter-Reihenfolge.

In [ ]:
def rank_johnson_trotter(pi,n):
    """Berechne die Nummer einer Permutationen von {1,2,...,n}
    in der Auflistung mit dem Johnson-Trotter-Algorithmus."""
    # Startwert für den Rückgabewert "rank"
    rank = 0
    # Länge des Permutationswortes sollte natürlich n sein
    if n != len(pi):
        print('Oops!')
    # Startwert für Rückgabewert "Liste b"
    # list_b = [0]*n
    # Inverse Permutation:
    invpi = invert_pi(pi)
    
    for i in range(1,n+1):
        # Wenn wir im Permutationswort pi alle Elemente > i wegstreichen,
        # an welcher Stelle steht dann i - Das ist äquivalent mit:
        # Wieviele Elemente j KLEINER i stehen VOR i im Permutationswort pi
        # Wir beantworten diese Frage mithilfe der INVERSEN Permutation:
        # Für wieviele Elemente j KLEINER i gilt pi^{-1}(j) < pi^{-1}(i)
        moves = count_less_than_last(invpi[:i])
        # Damit bestimmen wir nun "rekursiv" die gewünschte Nummer:
        if rank % 2 == 1:
            remainder = moves
        else:
            remainder = i-1-moves
        # list_b[i-1] = remainder
        rank = i*rank + remainder
    return rank # , list_b

In [ ]:
write_function_snippet(rank_johnson_trotter,
    **COMMON,
    the_caption=r'Rang (also Nummer) einer Permutation der $\symmgroup n$ in der Johnson--Trotter--Reihenfolge.',
    preamble = ''
)

## Unranking in der Johnson-Trotter-Reihenfolge.

In [ ]:
def unrank_johnson_trotter(num,n):
    """Berechne die Permutation von {1,2,...,n} mit Nummer n in der
    Auflistung mit dem Johnson-Trotter-Algorithmus."""
    # num muß zwischen 0 und n!-1 liegen:
    if num < 0 or num >= factorial(n):
        print("Sehr lustig ....")
        return []

    # Initialisiere Rückgabewert pi
    pi = [0]*n
    # Startwert für Rückgabewert "Liste b"
    # list_b = [0]*n
    # Hilfsgrößen P, R, C
    P = num
    for j in range(n,0,-1):
        R = P % j
        P = P // j
        # list_b[j-1] = R
        if P % 2 == 1:
            k = 0
            direction = 1
        else:
            k = n+1
            direction = -1
        
        C = 0
        while True:
            k = k + direction
            if pi[k-1] == 0:
                C+= 1
            if C > R:
                break
        pi[k-1] = j
        
    return pi # , list_b

In [ ]:
write_function_snippet(unrank_johnson_trotter,
    **COMMON,
    the_caption=r'Unranking in der Johnson--Trotter--Reihenfolge.',
    preamble = ''
)

## Generator für Zahl-Partitionen.
 Zahl-Partitionen von $n$ werden als _absteigend geordnete_ Folge der
 Teile (Summanden) angeschrieben (hier intern mit angehängten Nullen, die wir
 bei den Rückgabewerten weglassen). Die Funktion liefert einen _Iterator_,
 der diese Folgen in umgekehrter lexikographischer Ordnung erzeugt.

In [4]:
def generate_integer_partitions(n):
    """Generator, der einen Iterator liefert, der alle Partitionen
    von n erzeugt"""
    #@ Zahl--Partitionen von $n$ werden als \EM{absteigend geordnete} Folge der
    #@ Teile (Summanden) angeschrieben (hier intern mit angehängten Nullen, die wir
    #@ bei den Rückgabewerten weglassen). Die Funktion liefert einen \EM{Iterator},
    #@ der diese Folgen in umgekehrter lexikographischer Ordnung erzeugt.
    # Eine Partition von n kann höchstens n Teile haben: Wir bereiten ein
    # numpy-array dieser Länge vor, mit Datentyp int
    mu = np.zeros(n, dtype=int)
    # Die "kleinste" Partition (in umgekehrt lexikographischer Ordnung)
    # ist einfach gleich [n] (bzw. [n,0,0, ...,0]).
    mu[0] = n
    nof_parts = 1
    while True:
        # Der Befehl "yield" (statt "return") macht aus der Funktion einen
        # Generator:  Rückgabe ohne "trailing zeros".
        yield mu[:nof_parts]
        # Wenn eine Partition mit 1 beginnt, gibt es keine "größere" (in
        # umgekehrt lexikographischer Ordnung) mehr:
        if mu[0] == 1:
            break
        # Suche letzten Teil der Partition, der größer als 1 ist:
        # Dazu verwenden wir die "Vektor-Funktionen" von numpy;
        # der Ausdruck "mu>0" liefert ein boolean numpy-array
        # in dem wir die "nicht False"-Einträge abzählen, die
        # sind aber intern "nicht Null"-Einträge:
        nof_greater_one = np.count_nonzero(mu>1)
        # Dieser letzte Teil, der größer ist als 1, wird um 1 vermindert ...
        new_part = mu[nof_greater_one-1]-1
        # ... und der ursprüngliche Teil wird mit den restlichen Einsern
        # (sofern vorhanden) zusammengezählt ...
        sum_rest = np.sum(mu[nof_greater_one-1:])
        # ... aus dieser Summe machen wir "soviel wie möglich" Teile
        # new_part (Achtung: ganzzahlige Division "//" verwenden, sonst
        # ist das Ergebnis vom Typ float!) ...
        nof_new_parts = sum_rest//new_part
        # ... der Rest wird hinten als ein neuer Teil angestückelt ...
        additional_part = sum_rest - new_part*nof_new_parts
        # ... und zwar durch "Slicing" (Herausschneiden) eines "Stückes"
        # aus mu, dem dann ein konstanter Wert zugewiesen wird; zuerst
        # mal die Teile der Größe new_part:
        alpha = nof_greater_one-1
        omega = alpha + nof_new_parts
        mu[alpha:omega] = new_part
        nof_parts = omega
        # Dann der zusätzliche Teil:
        if omega < n:
            mu[omega] = additional_part
            # Wenn der zusätzliche Teil nicht Null ist, erhöht er die
            # Anzahl der Teile:
            if additional_part:
                nof_parts += 1
            # Zuguterletzt: Alles nach dem zusätzlichen Teil wird auf
            # Null gesetzt.
            mu[omega+1:] = 0

In [7]:
N = 10
for pi in generate_integer_partitions(N):
    if sum(pi) != N:
        print(pi)

In [ ]:
write_function_snippet(generate_integer_partitions,
    **COMMON,
    the_caption=r'Generator für Zahl--Partitionen.',
    preamble = 'import numpy as np'
)

## Austabellieren von $p(n,k)$ durch eine Rekursion.
 Hier wird die Rekursion $p(n,k) = \sum_{j=1}^k\sum_{i=1}^{[n/j]} p(n-i\cdot j,j-1)$
 verwendet; die Ergebnisse werden wieder in
 einem numpy-array gespeichert:
 Für größere $n$ ist das Verfahren
 _deutlich_ schneller als die "Brute--Force--Abzählung" aller
 Partitionen, die man mit ```generate_integer_partitions```
 erzeugt.

In [ ]:
def p_nk_by_recursion(nnn):
    """"Erzeuge eine Tabelle der Zahlen p(n,k) für 0<=n,k,<=nnn"""
    #@ Hier wird die Rekursion $p(n,k) = \sum_{j=1}^k\sum_{i=1}^{[n/j]} p(n-i\cdot j,j-1)$
    #@ verwendet; die Ergebnisse werden wieder in
    #@ einem numpy-array gespeichert:
    #@ Für größere $n$ ist das Verfahren
    #@ \EM{deutlich} schneller als die ``Brute--Force--Abzählung'' aller
    #@ Partitionen, die man mit \pythoncode{generate\_integer\_partitions}
    #@ erzeugt.
    pnk = np.zeros((nnn+1)**2, dtype=int).reshape((nnn+1,nnn+1))
    # Fülle Zeile 0 mit Einsern: Es gibt _eine_ Partition von 0,
    # nämlich die _leere_ Partition.
    pnk[0][:] = 1
    # Fülle Zeile 1 ab Spalte 1 mit Einsern: Für $n=1$ gibt es
    # nur eine Partition, und deren erster Teil ist 1>0.
    pnk[1][1:] = 1
    # Wir bestimmen alle Partitionen von n ...
    for n in range(2,nnn+1):
        # Mit größtem (also ersten) Teil kleinergleich k ...
        for k in range(1,n+1):
            # Durch eine Rekursion:
            for j in range(1,k+1):
                for i in range(1,n//j+1):
                    pnk[n][k]+= pnk[n-j*i,j-1]
        for k in range(n+1,nnn+1):
            pnk[n,k] = pnk[n,n]
    return pnk

In [ ]:
write_function_snippet(p_nk_by_recursion,
    **COMMON,
    the_caption=r'Austabellieren von $p\of{n,k}$ durch eine Rekursion.',
    preamble = 'import numpy as np'
)

## Ranking: Zahl-Partitionen $\pi$ in umgedrehter lexikographischer Ordnung.
 Unter Verwendung der Funktion $p(n,k)$ ist das Ranking
 einer Partition $\pi$ ganz leicht: Diese Funktion ist uns
 aber _nur rekursiv_ gegeben; hier wird sie in Form einer
 Tabelle berechnet.

In [ ]:
def rank_integer_partitions(pi):
    """Bestimme den Rang der Zahlpartition pi (gegeben als absteigend
    geordnetes numpy-array von ganzen Zahlen größer Null) in
    umgedrehter lexikographischer Ordnung"""
    #@ Unter Verwendung der Funktion $p\of{n,k}$ ist das Ranking
    #@ einer Partition $\pi$ ganz leicht: Diese Funktion ist uns
    #@ aber \EM{nur rekursiv} gegeben; hier wird sie in Form einer
    #@ Tabelle berechnet.
    # Anzahl der Teile:
    nof_parts=len(pi)
    # Bestimme die Partialsummen der Teile, mit Null beginnend
    local_pi_partial_sums = np.zeros(nof_parts+1, dtype=int)
    local_pi_partial_sums[1:] = np.cumsum(pi)
    n = local_pi_partial_sums[-1]
    # Berechne rekursiv die Funktionswerte p(n,k) in einer Tabelle
    pnk = p_nk_by_recursion(n)
    # Bestimme den Rang von pi
    rank = pnk[n][n] - 1
    # Obwohl pi und local_pi_partial_sums nicht gleich lang
    # sind, können wir trotzdem "zip" anwenden:
    for sum_pi_j, pi_j in zip(local_pi_partial_sums, pi):
        rank-=pnk[n-sum_pi_j][pi_j-1]
    return rank

In [ ]:
write_function_snippet(rank_integer_partitions,
    **COMMON,
    the_caption=r'Ranking: Zahl--Partitionen $\pi$ in umgedrehter lexikographischer Ordnung.',
    preamble = 'import numpy as np'
)

## Unranking: Zahl-Partitionen von $n$ in umgedrehter lexikographischer Ordnung.
 Unter Verwendung der Funktion $p(n,k)$ müssen wir die
 (eindeutige) Darstellung von rank als $\sum p(n_j, pi_j-1)$
 finden: $\pas{pi_1,pi_2,...}$ ist dann die gesuchte Zahlpartition.

In [ ]:
def unrank_integer_partitions(rank, n):
    """Bestimme die Zahlpartition von n mit Rang rank
    in umgedrehter lexikographischer Ordnung"""
    #@ Unter Verwendung der Funktion $p\of{n,k}$ müssen wir die
    #@ (eindeutige) Darstellung von rank als $\sum p(n_j, pi_j-1)$
    #@ finden: $\pas{pi_1,pi_2,...}$ ist dann die gesuchte Zahlpartition.
    # Berechne rekursiv die Funktionswerte p(n,k) in einer Tabelle
    pnk = p_nk_by_recursion(n)
    p_n = pnk[n][n]
    if rank < 0 or rank >= p_n:
        print(f'Rank {rank} not in [0,{p_n-1}]!')
        return None
    # Finde die eindeutige Zahlendarstellung von p(n)-1-rank als Summe
    # p(n_j, pi_j-1), wobei n_j = rank - pi_1 - p_2 - ... - p_{j-1}.
    pi = []
    num = p_n - rank - 1
    n_j = n
    while n_j:
        # Finde das größte pi_j sodaß p(n_j, pi_j - 1) <= num:
        # "Technisch" machen wir das durch die numpy-Funktion
        # "count_nonzero", die wir auf Zeile "pnk[n_j] <= num"
        # anwenden; das ist eine Zeile, die 1 enthält, wo die
        # Ungleichung erfüllt ist, und 0 sonst.
        pi_j = np.count_nonzero(pnk[n_j] <= num)
        # Wenn pi_j = 1, dann ist die restliche Partition schon
        # eindeutig bestimmt (als n_j Einser):
        if pi_j == 1:
            return np.array(pi + [1]*n_j)
        # Ansonsten: Update num, pi und n_j ...
        num-=  pnk[n_j][pi_j - 1]
        pi+=   [pi_j]
        n_j-=  pi_j
        # ... und weiter in der Schleife!

    return np.array(pi, dtype = int)

In [ ]:

write_function_snippet(unrank_integer_partitions,
    **COMMON,
    the_caption=r'Unranking: Zahl--Partitionen von $n$ in umgedrehter lexikographischer Ordnung.',
    preamble = 'import numpy as np'
)

## Erzeugung zufälliger Zahl-Partitionen.
 Wir verwenden die Bausteine zum Thema Zahl--Partitionen für
 einen "Zufalls-Zahlpartitionen-Generator": Die Tabelle mit
 den rekursiv berechneten Werten $p(n,k)$ berechnen wir hier
 natürlich nur _einmal_.

In [ ]:
def random_integer_partitions(n):
    """Generator, der einen Iterator liefert, der zufällig
    gewählte Partitionen von n erzeugt"""
    #@ Wir verwenden die Bausteine zum Thema Zahl--Partitionen für
    #@ einen ``Zufalls--Zahlpar\-ti\-ti\-o\-nen--Generator'': Die Tabelle mit
    #@ den rekursiv berechneten Werten $p\of{n,k}$ berechnen wir hier
    #@ natürlich nur \EM{einmal}.
    pnk = p_nk_by_recursion(n)
    p_n = pnk[n][n]
    print(f'Es gibt {p_n} Zahlpartitionen von {n}.')
    while True:
        # Wähle zufällig rank in [0, p_n-1] ...
        rank = np.random.randint(0, p_n)
        # ... und erzeuge die Zahlpartition
        # mit diesem Rang:
        pi = []
        num = p_n - rank - 1
        n_j = n
        while n_j:
            pi_j = np.count_nonzero(pnk[n_j] <= num)
            if pi_j == 1:
                pi+= [1]*n_j
                break
            num-=  pnk[n_j][pi_j - 1]
            pi+=   [pi_j]
            n_j-=  pi_j
        # Befehl 'yield' macht aus der Funktion einen Generator:
        yield np.array(pi)

In [ ]:
write_function_snippet(random_integer_partitions,
    **COMMON,
    the_caption=r'Erzeugung zufälliger Zahl--Partitionen.',
    preamble = 'import numpy as np'
)

## Monte-Carlo-Simulation zur Schätzung der durchschnittlichen Länge von Zahl--Partitionen.

In [ ]:
def estimate_average_length_integer_partitions(n,N):
    """Beispiel für Monte--Carlo--Simulation: Schätze durchschnittliche
    Länge der Zahlpartitionen von n durch Bestimmung der mittleren Länge
    von N zufällig gewählten Zahl--Partitionen."""
    sum_of_lengths = 0.0
    for i,pi in enumerate(random_integer_partitions(n)):
        sum_of_lengths += len(pi)
        if i == N-1:
            break
    
    print(f'Zahlpartitionen von {n} haben {sum_of_lengths/N} Teile')
    print(f'im Durchschnitt (geschätzt aus Random--Sample der Größe {N}).')

In [ ]:
write_function_snippet(estimate_average_length_integer_partitions,
    **COMMON,
    the_caption=r'Monte--Carlo--Simulation zur Schätzung der durchschnittlichen Länge von Zahl--Partitionen.',
    preamble = ''
)

## Visualisierung mit ```matplotlib```.
 Wenn man alle Ferrers Diagramme der Partitionen von n
 "übereinander schichtet", dann ergibt sich für jede Zelle
 im $n\times n$--Quadrat  eine "Höhe", die der
 Anzahl der Ferrers Diagramme entspricht, in denen diese Zelle enthalten
 ist): Mit den Visualisierungs-Tools in ```matplotlib```
 kann man das sehr hübsch sichtbar machen.

In [ ]:
def visualize_ferrers_diagram_heights(n):
    """Visualisiere: Anzahl der Ferrers Diagramme der
    Zahl--Partitionen von n, die die einzelnen 1 x 1-Boxen
    im n x n-Quadrat enthalten"""
    #@ Wenn man alle Ferrers Diagramme der Partitionen von n
    #@ ``übereinander schichtet'', dann ergibt sich für jede ``Box''
    #@ im $n\times n$--Quadrat  eine ``Höhe'' (entsprechend der
    #@ Anzahl der Ferrers Diagramme, in denen diese Box enthalten
    #@ ist): Mit den Visualisierungs-Tools in {\tt matplotlib}
    #@ kann man das sehr hübsch sichtbar machen.
    # Wir erzeugen zuerst die Daten durch simples Zählen:
    ferrers_box = np.zeros(n**2, dtype=float).reshape((n,n))
    for pi in generate_integer_partitions(n):
        for row, pi_j in enumerate(pi):
            # Zeilen numerieren wir hier "von unten", damit das
            # mit der graphischen Ausgabe zusammenpaßt:
            ferrers_box[n-1-row][:pi_j]+= 1.0
    # Wir dividieren alle Zahlen in dieser Matrix durch die Anzahl
    # aller Zahlpartitionen von n (die steht im Eintrag (n-1,0)):
    p_n = ferrers_box[n-1][0]
    ferrers_box/= p_n
    # Dann verwenden wir eine der (sehr zahlreichen!) Darstellungs-
    # möglichkeiten von matplotlib: Wir wollen eine einfache Abbildung
    # (figure) mit einem einzigen Achsenkreuz (axes):
    figure, axes = plt.subplots()
    # Die "Höhen" visualisieren wir durch Farben, die hier automatisch
    # gewählt werden:
    axes.pcolormesh(ferrers_box, shading='auto')
    # Um ein quadratisches Bild zu erhalten, das ja unserer Situation
    # entspricht, setzen wir die "aspect ratio" der Graphik auf 'equal':
    axes.set_aspect('equal')

    # Nun zeigen wir die Graphik:
    plt.title(f'Häufigkeit Kästchen in \n{(int(p_n))} Ferrers Diagrammen:')
    plt.show()

In [ ]:
write_function_snippet(visualize_ferrers_diagram_heights,
    **COMMON,
    the_caption=r'Visualisierung mit {\tt matplotlib}.',
    preamble = """import numpy as np
import matplotlib.pyplot as plt"""
)

## Generator: Funktionen von beschränktem Wachstum (in Bijektion mit Mengen-Partitionen).
 Die Funktion f wird als numpy-Vektor (f(1), f(2), ... f(n)) codiert,
 ebenso die "Wachstumsbeschränkung". Durch Verwendung von numpy-Funktionalität
 wird der Code wieder recht übersichtlich.

In [ ]:
def functions_of_restricted_growth(n):
    """Generator, der einen Iterator liefert, der alle Funktionen
    von beschränktem Wachstum auf [n] in lexikographischer Ordnung
    liefert."""
    #@ Die Funktion f wird als numpy-Vektor (f(1), f(2), ... f(n)) codiert,
    #@ ebenso die "Wachstumsbeschränkung". Durch Verwendung von numpy-Funktionalität
    #@ wird der Code wieder recht übersichtlich.
    the_func = np.ones(n, dtype=int)
    the_max  = np.ones(n, dtype=int)
    if n>1:
        the_max[1:]+= 1
    # the_max is a non-decreasing pointwise upper bound for the_func;
    # and the sequence of the_max-es is "pointwise non-decreasing":
    while True:
        yield the_func
        # For which indices is the_max > the_func? The numpy-function
        # flatnonzero(a) returns the indices that are non-zero in the
        # flattened version of a. (Flattened heißt hier: das array a
        # wird "als Vektor interpretiert"; eine Matrix [[1,2],[3,4]]
        # also z.B. als [1,2,3,4] - das ist genau das, was wir brauchen.)
        indices_of_possible_increase = np.flatnonzero(the_max-the_func)
        if len(indices_of_possible_increase):
            # Choose the last of these possible indices ...
            index_to_increase = indices_of_possible_increase[-1]
            # ... and increase the function at this index ...
            the_func[index_to_increase]+=1
            # ... and set the function at all later indices to 1:
            the_func[index_to_increase+1:] = 1
            # Update the_max, if necessary:
            if the_max[index_to_increase] == the_func[index_to_increase]:
                the_max[index_to_increase+1:] = the_max[index_to_increase]+1
        else:
            break

In [ ]:
write_function_snippet(functions_of_restricted_growth,
    **COMMON,
    the_caption=r'Generator: Funktionen von beschränktem Wachstum (in Bijektion mit Mengen--Partitionen).',
    preamble = 'import numpy as np'
)

## Rekursive Berechnung der $r$-Bell Zahlen.
 Die Matrix, die wir hier berechnen, hat in der OEIS-Datenbank
 Index A095148.

In [ ]:
def compute_r_Bell_numbers(nnn):
    """Berechne die r--Bell Zahlen, die bei der Bestimmung
    des Rangs einer Funktion von beschränktem Wachstum in
    lexikographischer Ordnung auftreten"""
    #@ Die Matrix, die wir hier berechnen, hat in der OEIS-Datenbank
    #@ Index A095148.
    # Vorbereitung der Matrix als numpy-array von ganzen Zahlen
    result = np.zeros(nnn**2, dtype=int).reshape((nnn,nnn))
    # Die letzte und vorletzte Zeile ergeben sich aus der
    # Anfangsbedingung der Rekurion
    result[-1] = 1
    result[-2][:-1] = np.arange(2,nnn+1)
    # Die übrigen Einträge bestimmen wir aus der Rekursionsgleichung
    for row in range(nnn-3,-1,-1):
        for col in range(row+1):
            new_b = (col+1)*result[row+1][col] + result[row+1][col+1]
            result[row][col] = new_b

    return result

In [ ]:
write_function_snippet(compute_r_Bell_numbers,
    **COMMON,
    the_caption=r'Rekursive Berechnung der $r$--Bell Zahlen.',
    preamble = 'import numpy as np'
)

## Ranking: Funktionen von beschränktem Wachstum (in lexikographischer Ordnung).
 Rangfunktion: $\text{function} f\mapsto\text{rank}$.
 Wir brauchen hier als Vorbereitung eine Matrix von
 Zahlen, aus der wir dann Werte ablesen, die wir in
 der Berechnung verwenden.


In [ ]:
def rank_function_of_restricted_growth(the_func):
    """Berechne den Rang einer Funktion von beschränktem
    Wachstum in lexikographischer Ordnung."""
    #@ Rangfunktion: $\text{function} f\mapsto\text{rank}$.
    #@ Wir brauchen hier als Vorbereitung eine Matrix von
    #@ Zahlen, aus der wir dann Werte ablesen, die wir in
    #@ der Berechnung verwenden.
   # Vorbereitung: r-Bell-Zahlen und Funktion, die sie "ausliest".
    nnn = len(the_func)
    the_table = compute_r_Bell_numbers(nnn)
    def local_d(i,m):
        return the_table[i-1][m-1]
    # Wir bilden die "abschnittsweisen Maxima" von the_func, also
    #       [ max(the_func[:i]) for i in range(1, nnn + 1) ];
    # mit numpy-Funktionalität können wir das auch so erledigen:
    the_maxima = np.roll(np.maximum.accumulate(the_func),1)
    # (numpy.roll(a, k) "vertauscht zyklisch" nach rechts um k Stellen;
    # daß nun auf the_maxima[0] ein "falscher Wert" steht, macht hier
    # nichts, weil der in der folgenden Summe ohnehin nicht vorkommt.)
    
    # Berechne den Rang aus der Funktion als Summe:
    rank = 0
    for i, m_i, f_i in zip(range(2,nnn+1), the_maxima[1:], the_func[1:]):
        rank+= (f_i-1)*local_d(i,m_i)
    
    return rank

In [ ]:
write_function_snippet(rank_function_of_restricted_growth,
    **COMMON,
    the_caption=r'Ranking: Funktionen von beschränktem Wachstum (in lexikographischer Ordnung).',
    preamble = 'import numpy as np'
)

## Unranking: Funktionen von beschränktem Wachstum aus dem Rang (in lexikographischer Ordnung).
 Umkehrung der Rangfunktion: $\text{rank}\mapsto\text{function} f$.
 Wir brauchen hier als Vorbereitung eine Matrix von
 Zahlen, aus der wir dann Werte ablesen, die wir in
 der Berechnung verwenden.


In [ ]:
def unrank_function_of_restricted_growth(rank, nnn):
    """Berechne die Funktion von beschränktem Wachstum, die in
    lexikographischer Ordnung den gegebene Rang hat."""
    #@ Umkehrung der Rangfunktion: $\text{rank}\mapsto\text{function} f$.
    #@ Wir brauchen hier als Vorbereitung eine Matrix von
    #@ Zahlen, aus der wir dann Werte ablesen, die wir in
    #@ der Berechnung verwenden.
    # Vorbereitung: r-Bell-Zahlen und Funktion, die sie "ausliest".
    the_table = compute_r_Bell_numbers(nnn)
    def local_d(i,m):
        return the_table[i-1][m-1]
    
    # Vorbereitung der Arrays:
    the_func = np.zeros(nnn, dtype=int)
    the_maxima = np.zeros(nnn, dtype=int)
    # Die Funktion hat f(1) = 1, und das max({f(1}) ist ebenso 1:
    the_func[0] = the_maxima[0] = 1    
    
    # Berechne die Funktion aus dem Rang:
    for i in (range(1,nnn)):
        m = the_maxima[i-1]
        # Achtung: Indices in Python beginnen bei 0
        d_im = local_d(i+1,m)
        # Schauen wir mal, ob the_func[i] = m+1 möglich ist:
        m_d_im = m*d_im
        if m_d_im <= rank:
            # Wenn ja: Setze the_func[i] und neues Maximum:
            the_func[i] = the_maxima[i] = m+1
            rank-= m_d_im
        else:
            # Wenn nein: Berechne the_func[i] durch Division mit Rest ...
            the_func[i] = rank//d_im + 1
            rank%= d_im
            # ... und das alte Maximum wird "weiterverwendet":
            the_maxima[i] = m
    
    return the_func

In [ ]:
write_function_snippet(unrank_function_of_restricted_growth,
    **COMMON,
    the_caption=r'Unranking: Funktionen von beschränktem Wachstum aus dem Rang (in lexikographischer Ordnung).',
    preamble = 'import numpy as np'
)

## Abfrage der OEIS-Datenbank mit ```pycurl```.
 Die _Online Ecyclopedia of Integer Sequences_ ist eine
 nützliche Quelle, um (noch) unbekannte Folgen ganzer
 Zahlen zu identifizieren. Man kann entsprechende Anfragen
 natürlich händisch in einem Web-Browser eingeben, aber
 wir illustrieren hier einmal die Internet--Fähigkeiten von
 Python.

In [ ]:
def OEIS_query(sequence, n=5):
    """Abfrage der OEIS-Datenbank"""
    #@ Die \EM{Online Ecyclopedia of Integer Sequences} ist eine
    #@ nützliche Quelle, um (noch) ``unbekannte'' Folgen ganzer
    #@ Zahlen zu identifizieren. Man kann entsprechende Anfragen
    #@ natürlich manuell in einem Web--Browser eingeben, aber
    #@ wir illustrieren hier einmal die Internet--Fähigkeiten von
    #@ Python.

    # Eine "lokale" Klasse, die sich um die Antwort aus dem Internet
    # kümmert:
    class ResponseManager:
        def __init__(self):
            self.contents = ""
        def callback(self, buf):
            # Das Internet liefert einen "Byte-String", den wir
            # in einen "normalen" String decodieren (UTF-8 ist
            # ein sehr gebräuchliches Format zum Codieren von
            # Texten):
            self.contents += buf.decode('utf-8')

    # Zusammenstellen der HTML-Anfrage
    querystr = "http://oeis.org/search?q="
    querystr += ",".join([str(x) for x in sequence])
    querystr += "&n=" + str(n) + "&fmt=text"

    # Anfrage absetzen, Antworten speichern
    t = ResponseManager()
    c = pycurl.Curl()
    c.setopt(c.URL, querystr)
    c.setopt(c.WRITEFUNCTION, t.callback)
    c.perform()
    c.close()

    # Antwort zurückgeben (In Form eines Strings)
    return t.contents

In [ ]:
write_function_snippet(OEIS_query,
    **COMMON,
    the_caption=r'Abfrage der OEIS--Datenbank mit pycurl.',
    preamble = 'import pycurl'
)

## Extraktion der Informationen aus der Antwort der OEIS-Datenbank.
 Die Antwort von OEIS besteht aus einem langen String, der
 sehr einfach aufgebaut ist und aus dem man die Bedeutung
 der Informationen rasch extrahieren kann.

In [ ]:
#Dictionary der "Codes" auf einer OEIS--Seite
OEIS_PARSER = {
    'I': 'OEIS Index',
    'S' : 'Series',
    'T' : 'Series',
    'U' : 'Series',
    'F' : 'Formula',
    'e' : 'Example',
    't' : 'Mathematica',
    'Y' : 'Cross references',
    'K' : 'Keywords',
    'A' : 'Author'
}
# Menge der Werte in OEIS_PARSER
OEIS_VALUES = set(OEIS_PARSER.values())

def OEIS_parse(response):
    """Sehr einfacher 'Parser', der die Antwort der
    OEIS Datenbank verarbeitet"""
    #@ Die Antwort von OEIS besteht aus einem langen String, der
    #@ sehr einfach aufgebaut ist und aus dem man die Bedeutung
    #@ der Informationen rasch extrahieren kann.
    # Dictionary für unsere formatierte Ausgabe
    formatted_responses = dict()
    for line in response.split("\n"):
        # Die Zeilen, die wir verarbeiten wollen, beginnen alle mit "%"
        # und einem Buchstaben, der die Bedeutung der restlichen Zeile
        # bestimmt (siehe dictionary OEIS_PARSER); Achtung: Es kann auch
        # leere Zeilen geben!
        if not line or line[0] != '%':
            continue
        # Wir schauen uns den zweiten Buchstaben an:
        second_character = line[1]
        # Anstatt vorher zu schauen, ob dieser zweite
        # Buchstabe im dictionary OEIS_PARSER vorkommt,
        # probieren wir das einfach ...
        try:
            record = OEIS_PARSER[second_character]
            data = line[3:]
        # ... und "fangen" die Fehlermeldung ab, wenn's schiefgeht:
        except KeyError:
            continue
        if record == 'OEIS Index':
            # Wir basteln ein neues dictionary für die gefundene
            # Zahlenfolge, die mit data bezeichnet ist:
            OEIS_INDEX = data
            formatted_responses[OEIS_INDEX] = dict(
                zip(
                    OEIS_VALUES,
                    ['']*len(OEIS_VALUES)
                )
            )
            formatted_responses[OEIS_INDEX][record]+= OEIS_INDEX
        else:
            # Die ersten 8 Buchstaben sind der aktuelle OEIS_INDEX und
            # ein Space; die lassen wir weg:
            formatted_responses[OEIS_INDEX][record]+=(data[8:]+'\n')
            
    return formatted_responses

In [ ]:
write_function_snippet(OEIS_parse,
    **COMMON,
    the_caption=r'Extraktion der Informationen aus der Antwort der OEIS--Datenbank.',
    preamble = ''
)